**예시 코드_IMDB sentiment analysis using LlaMa3.2**

예시 코드를 바탕으로 아래 과제 수행 필요
1. NSMC(네이버 영화 리뷰 감성분석 데이터셋)에 대해 fine-tuning 수행 (40점)
2. LoRA 설정을 바꿔서 성능 테스트 (30점)
3. Few-shot 프롬프트 적용 (30점)

최종적으로 총 4가지 경우에 대한 결과가 필요함
1. 학습을 전혀 하지 않은 Baseline 모델 결과
2. r=4로 fine-tuning한 결과
3. 다른 r값으로 fine-tuning한 결과
4. Few-shot 프롬프트를 적용한 결과(r값은 위의 2,3 둘 중 하나에 해당하면 됨)

In [ ]:
#관련 라이브러리 설치

!pip install datasets unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.0/175.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 76.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.7/68.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import datasets
import torch
from trl import SFTTrainer
import pandas as pd
import numpy as np
import os
import pandas as pd
import numpy as np
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer
import warnings

#하이퍼파라미터 설정 및 Llama3.2 로드
max_seq_length = 2048
dtype = None

model_name = "unsloth/Llama-3.2-1B-Instruct";load_in_4bit = True,

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)

dataset = load_dataset("e9t/nsmc")

processed_dataset = dataset.remove_columns("id")
processed_dataset = processed_dataset.rename_column("document", "text")



#속도 이슈로 데이터셋 샘플링 (NSMC에서도 학습 5000개, 테스트 1000개로 샘플링 요망)
sampled_train = processed_dataset['train'].shuffle(3407).select(range(5000))
sampled_test = processed_dataset['test'].shuffle(3407).select(range(1000))

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

...

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

The repository for e9t/nsmc contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/e9t/nsmc.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#감성분석 스코어를 측정하기 위해 Yes, No 토큰의 어휘집 내 index 파악
yes_token_id = tokenizer.encode("Yes", add_special_tokens=False)[0]
no_token_id = tokenizer.encode("No", add_special_tokens=False)[0]

In [ ]:
print("trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))

trainable parameters: 67584


In [ ]:
#LoRA 설정
from peft import LoftQConfig

model_with_r4 = FastLanguageModel.get_peft_model(
    model,
    r = 4,
    target_modules = ["q_proj", "v_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
)
print("trainable parameters:", sum(p.numel() for p in model_with_r4.parameters() if p.requires_grad))

Not an error, but Unsloth cannot patch MLP layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch Attention layers with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Not an error, but Unsloth cannot patch O projection layer with our manual autograd engine since either LoRA adapters
are not enabled or a bias term (like in Qwen) is used.
Unsloth 2024.12.8 patched 16 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


trainable parameters: 425984


In [ ]:
#감성분석 프롬프트 템플릿
prompt = """Here is a movie review:
{}

Does this tweet have a positive sentiment? Answer with "Yes" or "No".

SOLUTION
The correct answer is: "{}"""

positivelabel = "Yes"
negativelabel = "No"

#IMDB 데이터 -> 프롬프트 변환을 위한 함수
def formatting_prompts_func(dataset_):
    if isinstance(dataset_['text'], str):
        if model_name.lower().__contains__("qwen"):
            return [""]*100
        elif model_name.lower().__contains__("llama"):
            return " "
        else:
            return " "

    texts = []
    for i in range(len(dataset_['text'])):
        t = dataset_['text'][i]
        label = positivelabel if dataset_['label'][i] == 1 else negativelabel
        text = prompt.format(t, label)
        texts.append(text)
    return texts

In [ ]:
#SFT 학습용 객체 설정. 현재 작업은 분류이지만 fine-tuning은 언어 모델링에 대해 수행될 예정임. 그러나 실제 레이블에 따른 Yes/No 대답을 학습하므로 이후 성능 향상을 확인할 수 있음
trainer = SFTTrainer(
    model = model_with_r4,
    tokenizer = tokenizer,
    train_dataset = sampled_train,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 32,   #Batch size
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1,
        report_to = "none",
        group_by_length = False,
    ),
    formatting_func=formatting_prompts_func,
)

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
from collections import defaultdict
import torch.nn.functional as F
from tqdm import tqdm

#추론을 위한 함수 구현
def test_model(model, tokenizer, test_dataset):
  #Manual tokenizing
  tokenized_inputs = []
  for i in tqdm(range(len(test_dataset['text']))):
      text = test_dataset['text'][i]
      test_str = prompt.format(text, "")
      tokenized_input = tokenizer(test_str, return_tensors="pt", add_special_tokens=False)
      tokenized_inputs.append((tokenized_input, test_str, test_dataset['label'][i]))

  tokenized_inputs.sort(key=lambda x: x[0]['input_ids'].shape[1])

  grouped_inputs = defaultdict(list)
  for tokenized_input, test_str, label in tokenized_inputs:
      length = tokenized_input['input_ids'].shape[1]
      grouped_inputs[length].append((tokenized_input, test_str, label))

  batch_size = 32
  all_outputs = []
  all_strings = []
  all_labels = []

  #추론 루프
  for length, group in tqdm(grouped_inputs.items()):
      for i in range(0, len(group), batch_size):
          batch = group[i:i+batch_size]
          batch_inputs = [item[0] for item in batch]
          batch_strings = [item[1] for item in batch]
          batch_labels = [item[2] for item in batch]

          # Concatenate the batch inputs
          input_ids = torch.cat([item['input_ids'] for item in batch_inputs], dim=0).to("cuda")
          attention_mask = torch.cat([item['attention_mask'] for item in batch_inputs], dim=0).to("cuda")

          # Forward pass
          with torch.no_grad():
              outputs = model(input_ids=input_ids, attention_mask=attention_mask)

          logits = outputs.logits  # [batch_size, seq_len, vocab_size]

          # 시퀀스의 마지막 토큰 위치 로짓 선택 (필요 시 다른 위치 사용 가능)
          last_token_logits = logits[:, -1, :]  # [batch_size, vocab_size]

          # yes_token_id와 no_token_id에 해당하는 로짓만 추출
          yes_no_logits = last_token_logits[:, [no_token_id, yes_token_id]]  # [batch_size, 2]

          # softmax 적용 (model의 출력에는 어휘집 내 모든 토큰에 대한 logit 값이 있지만, 그 중에 Yes, No에 대한 값만 추출하여 softmax를 적용해서 분류를 수행)
          probabilities = torch.softmax(yes_no_logits, dim=-1)  # [batch_size, 2]

          predictions = torch.argmax(probabilities, dim=-1)

          all_outputs.extend(predictions.cpu().numpy())
          all_labels.extend(batch_labels)
          all_strings.extend(batch_strings)

  # 성능 측정
  correct = 0
  total = 0

  for i in range(len(all_outputs)):
      pred = str(all_outputs[i])
      label = str(all_labels[i])

      if pred == label:
          correct += 1
      total += 1

  print(f"Correct: {correct} Total: {total} Accuracy: {correct/total}")

#Fine-tuning 전의 성능 확인
test_model(model, tokenizer, sampled_test)

100%|██████████| 100/100 [00:10<00:00,  9.09it/s]

Correct: 483 Total: 1000 Accuracy: 0.483


**baseline 성능 Correct: 483 Total: 1000 Accuracy: 0.483**

In [ ]:
#Fine-tuning
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 157
 "-____-"     Number of trainable parameters = 425,984


Step,Training Loss
1,4.858200
2,4.780400
3,5.017900
4,4.727800
5,4.954000
6,4.826100
7,4.663600
8,4.819800
9,4.491700
10,4.508800


In [ ]:
#Fine-tuning 후의 성능 확인
test_model(model_with_r4, tokenizer, sampled_test)

100%|██████████| 100/100 [00:06<00:00, 16.16it/s]

Correct: 731 Total: 1000 Accuracy: 0.731


**R4 성능 Correct: 731 Total: 1000 Accuracy: 0.731**

In [ ]:
#LoRA 설정
from peft import LoftQConfig

#하이퍼파라미터 설정 및 Llama3.2 로드
max_seq_length = 2048
dtype = None

model_name = "unsloth/Llama-3.2-1B-Instruct";load_in_4bit = True,

model2, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)
print("trainable parameters:", sum(p.numel() for p in model2.parameters() if p.requires_grad))

model_with_r2 = FastLanguageModel.get_peft_model(
    model2,
    r = 2,
    target_modules = ["q_proj", "v_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
)
print("trainable parameters:", sum(p.numel() for p in model_with_r2.parameters() if p.requires_grad))

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
trainable parameters: 67584
trainable parameters: 212992


In [ ]:
#SFT 학습용 객체 설정. 현재 작업은 분류이지만 fine-tuning은 언어 모델링에 대해 수행될 예정임. 그러나 실제 레이블에 따른 Yes/No 대답을 학습하므로 이후 성능 향상을 확인할 수 있음
trainer2 = SFTTrainer(
    model = model_with_r2,
    tokenizer = tokenizer,
    train_dataset = sampled_train,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 32,   #Batch size
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1,
        report_to = "none",
        group_by_length = False,
    ),
    formatting_func=formatting_prompts_func,
)

In [ ]:
#Fine-tuning
trainer_stats = trainer2.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 157
 "-____-"     Number of trainable parameters = 212,992


Step,Training Loss
1,4.858200
2,4.780400
3,5.018200
4,4.733400
5,4.961900
6,4.846300
7,4.697800
8,4.865200
9,4.556300
10,4.586600


In [ ]:
#Fine-tuning 후의 성능 확인
test_model(model_with_r2, tokenizer, sampled_test)

100%|██████████| 100/100 [00:06<00:00, 15.45it/s]

Correct: 728 Total: 1000 Accuracy: 0.728


**R2 성능 Correct: 728 Total: 1000 Accuracy: 0.728**

In [ ]:
#LoRA 설정
from peft import LoftQConfig

#하이퍼파라미터 설정 및 Llama3.2 로드
max_seq_length = 2048
dtype = None

model_name = "unsloth/Llama-3.2-1B-Instruct";load_in_4bit = True,

model3, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,load_in_4bit = load_in_4bit,
    max_seq_length = max_seq_length,
    dtype = dtype,
)
print("trainable parameters:", sum(p.numel() for p in model3.parameters() if p.requires_grad))

model_with_r4fs = FastLanguageModel.get_peft_model(
    model3,
    r = 4,
    target_modules = ["q_proj", "v_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = True,
)
print("trainable parameters:", sum(p.numel() for p in model_with_r4fs.parameters() if p.requires_grad))

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 8.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
trainable parameters: 67584
trainable parameters: 425984


In [ ]:
#SFT 학습용 객체 설정. 현재 작업은 분류이지만 fine-tuning은 언어 모델링에 대해 수행될 예정임. 그러나 실제 레이블에 따른 Yes/No 대답을 학습하므로 이후 성능 향상을 확인할 수 있음
trainer3 = SFTTrainer(
    model = model_with_r4fs,
    tokenizer = tokenizer,
    train_dataset = sampled_train,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 32,   #Batch size
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        learning_rate = 1e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        num_train_epochs = 1,
        report_to = "none",
        group_by_length = False,
    ),
    formatting_func=formatting_prompts_func,
)

In [ ]:
# Demonstrations 추가
positivelabel = "Yes"
negativelabel = "No"

prompt = """Here are some examples of movie reviews and their sentiment:
Review: "마치 미국애니에서 튀어나온듯한 창의력없는 로봇디자인부터가,고개를 젖게한다"
Sentiment: positivelabel

Review: "괜찮네요오랜만포켓몬스터잼밌어요"
Sentiment: negativelabel

Here is a new movie review:
{}
Does this review have a positive sentiment? Answer with "Yes" or "No".

SOLUTION
The correct answer is: "{}"""

In [ ]:
#Fine-tuning
trainer_stats = trainer3.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 5,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 157
 "-____-"     Number of trainable parameters = 425,984


Step,Training Loss
1,4.858200
2,4.780400
3,5.016300
4,4.729700
5,4.952600
6,4.825500
7,4.663200
8,4.819700
9,4.489500
10,4.507600


In [ ]:
#Fine-tuning 후의 성능 확인
test_model(model_with_r4fs, tokenizer, sampled_test)

100%|██████████| 99/99 [00:07<00:00, 13.97it/s]

Correct: 750 Total: 1000 Accuracy: 0.75


**R4에서 fs 성능 Correct: 750 Total: 1000 Accuracy: 0.750**

정리

baseline 성능 Correct: 483 Total: 1000 Accuracy: 0.483

R4 성능 Correct: 731 Total: 1000 Accuracy: 0.731

R2 성능 Correct: 728 Total: 1000 Accuracy: 0.728

R4에서 fs 성능 Correct: 750 Total: 1000 Accuracy: 0.750